## Introduction 

Tatalakayin sa araling ito: 
- Ano ang pagtawag ng function at ang mga gamit nito 
- Paano gumawa ng pagtawag ng function gamit ang OpenAI 
- Paano isama ang pagtawag ng function sa isang aplikasyon 

## Learning Goals 

Pagkatapos makumpleto ang araling ito, malalaman mo kung paano at mauunawaan ang: 

- Layunin ng paggamit ng pagtawag ng function 
- Pagsasaayos ng Function Call gamit ang OpenAI Service 
- Paggawa ng epektibong pagtawag ng function para sa gamit ng iyong aplikasyon 


## Pag-unawa sa Pagtawag ng Function

Para sa araling ito, nais nating bumuo ng isang tampok para sa aming education startup na nagpapahintulot sa mga gumagamit na gumamit ng chatbot upang makahanap ng mga teknikal na kurso. Magrerekomenda kami ng mga kurso na angkop sa kanilang antas ng kasanayan, kasalukuyang tungkulin, at teknolohiyang interesado sila.

Upang makumpleto ito gagamitin natin ang kombinasyon ng:
 - `OpenAI` upang lumikha ng karanasan sa chat para sa gumagamit
 - `Microsoft Learn Catalog API` upang tulungan ang mga gumagamit na makahanap ng mga kurso base sa kahilingan ng gumagamit
 - `Function Calling` upang kunin ang query ng gumagamit at ipadala ito sa isang function upang gawin ang API request.

Upang magsimula, tingnan natin kung bakit nais nating gamitin ang function calling sa unang pagkakataon:

print("Mga mensahe sa susunod na request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # kumuha ng bagong tugon mula sa GPT kung saan makikita nito ang tugon ng function


print(second_response.choices[0].message)


### Bakit Function Calling

Kung nakatapos ka na ng anumang ibang aralin sa kursong ito, malamang na nauunawaan mo ang kapangyarihan ng paggamit ng Large Language Models (LLMs). Sana ay nakikita mo rin ang ilan sa kanilang mga limitasyon.

Ang Function Calling ay isang tampok ng OpenAI Service na idinisenyo upang tugunan ang mga sumusunod na hamon:

Hindi Konsistenteng Pormat ng Tugon:
- Bago ang function calling, ang mga tugon mula sa isang malaking language model ay hindi nakaayos at hindi konsistente. Kailangan ng mga developer na magsulat ng kumplikadong validation code upang hawakan ang bawat pagkakaiba sa output.

Limitadong Integrasyon sa Panlabas na Data:
- Bago ang tampok na ito, mahirap isama ang data mula sa ibang bahagi ng isang aplikasyon sa konteksto ng chat.

Sa pamamagitan ng pag-standardize ng mga pormat ng tugon at pagpapahintulot ng tuloy-tuloy na integrasyon sa panlabas na data, pinapasimple ng function calling ang pag-develop at binabawasan ang pangangailangan para sa karagdagang validation logic.

Hindi makakuha ang mga user ng mga sagot tulad ng "Ano ang kasalukuyang panahon sa Stockholm?". Ito ay dahil limitado ang mga modelo sa oras kung kailan sila sinanay.

Tingnan natin ang halimbawa sa ibaba na nagpapakita ng problemang ito:

Sabihin nating gusto nating gumawa ng database ng datos ng mga estudyante upang makapag-suggest ng tamang kurso para sa kanila. Sa ibaba ay may dalawang paglalarawan ng mga estudyante na halos magkapareho ang datos na nilalaman.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Nais naming ipadala ito sa isang LLM upang i-parse ang data. Maaari itong magamit sa aming aplikasyon upang ipadala ito sa isang API o itago sa isang database.

Gumawa tayo ng dalawang magkaparehong prompt na magtuturo sa LLM kung anong impormasyon ang nais naming makuha:


Gusto naming ipadala ito sa isang LLM upang suriin ang mga bahagi na mahalaga sa aming produkto. Kaya maaari kaming gumawa ng dalawang magkaparehong prompt upang utusan ang LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Pagkatapos likhain ang dalawang prompt na ito, ipapadala natin ang mga ito sa LLM gamit ang `openai.ChatCompletion`. Itinatago natin ang prompt sa variable na `messages` at itinalaga ang role bilang `user`. Ito ay upang gayahin ang isang mensahe mula sa isang user na isinusulat sa isang chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Ngayon maaari na nating ipadala ang parehong mga kahilingan sa LLM at suriin ang tugon na natanggap natin.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Kahit na pareho ang mga prompt at magkatulad ang mga paglalarawan, maaari tayong makakuha ng iba't ibang format ng property na `Grades`.

Kung patakbuhin mo ang cell sa itaas nang maraming beses, ang format ay maaaring `3.7` o `3.7 GPA`.

Ito ay dahil ang LLM ay kumukuha ng hindi nakaayos na data mula sa anyo ng nakasulat na prompt at nagbabalik din ng hindi nakaayos na data. Kailangan nating magkaroon ng nakaayos na format upang malaman natin kung ano ang aasahan kapag iniimbak o ginagamit ang data na ito.

Sa pamamagitan ng paggamit ng functional calling, masisiguro nating makakatanggap tayo ng nakaayos na data pabalik. Kapag gumagamit ng function calling, ang LLM ay hindi talaga tumatawag o nagpapatakbo ng anumang mga function. Sa halip, gumagawa tayo ng isang istruktura para sundan ng LLM sa mga tugon nito. Pagkatapos ay ginagamit natin ang mga nakaayos na tugon na iyon upang malaman kung anong function ang dapat patakbuhin sa ating mga aplikasyon.


![Daloy ng Tawag ng Function](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.tl.png)


Maaari nating kunin ang ibinalik mula sa function at ipadala ito pabalik sa LLM. Ang LLM ay sasagot gamit ang natural na wika upang tugunan ang tanong ng gumagamit.


### Mga Gamit ng paggamit ng function calls

**Pagtawag ng Panlabas na Mga Kasangkapan**  
Magaling ang mga chatbot sa pagbibigay ng mga sagot sa mga tanong mula sa mga gumagamit. Sa pamamagitan ng paggamit ng function calling, maaaring gamitin ng mga chatbot ang mga mensahe mula sa mga gumagamit upang tapusin ang ilang mga gawain. Halimbawa, maaaring hilingin ng isang estudyante sa chatbot na "Magpadala ng email sa aking instruktor na nagsasabing kailangan ko ng karagdagang tulong sa paksang ito". Maaari itong gumawa ng function call sa `send_email(to: string, body: string)`

**Gumawa ng API o Database Queries**  
Maaaring maghanap ang mga gumagamit ng impormasyon gamit ang natural na wika na kino-convert sa isang naka-format na query o API request. Isang halimbawa nito ay ang isang guro na nagtatanong "Sino ang mga estudyanteng nakatapos ng huling takdang-aralin" na maaaring tumawag ng function na pinangalanang `get_completed(student_name: string, assignment: int, current_status: string)`

**Paglikha ng Istrakturadong Data**  
Maaaring kunin ng mga gumagamit ang isang bloke ng teksto o CSV at gamitin ang LLM upang kunin ang mahahalagang impormasyon mula rito. Halimbawa, maaaring i-convert ng isang estudyante ang isang artikulo sa Wikipedia tungkol sa mga kasunduan sa kapayapaan upang gumawa ng mga AI flash cards. Magagawa ito sa pamamagitan ng paggamit ng function na tinatawag na `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Paglikha ng Iyong Unang Tawag sa Function

Ang proseso ng paglikha ng tawag sa function ay may 3 pangunahing hakbang:
1. Tawagan ang Chat Completions API gamit ang listahan ng iyong mga function at isang mensahe mula sa user
2. Basahin ang tugon ng modelo upang magsagawa ng aksyon tulad ng pagpapatupad ng isang function o API Call
3. Gumawa ng isa pang tawag sa Chat Completions API gamit ang tugon mula sa iyong function upang gamitin ang impormasyong iyon sa paggawa ng tugon para sa user.


![Daloy ng Tawag ng Function](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.tl.png)


### Mga Elemento ng isang pagtawag ng function

#### Input ng mga Gumagamit

Ang unang hakbang ay gumawa ng mensahe ng gumagamit. Maaari itong itakda nang dinamiko sa pamamagitan ng pagkuha ng halaga mula sa isang text input o maaari kang magtalaga ng halaga dito. Kung ito ang iyong unang beses na gumamit ng Chat Completions API, kailangan nating tukuyin ang `role` at ang `content` ng mensahe.

Ang `role` ay maaaring `system` (gumagawa ng mga patakaran), `assistant` (ang modelo) o `user` (ang end-user). Para sa pagtawag ng function, itatalaga natin ito bilang `user` at isang halimbawa ng tanong.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Paglikha ng mga function.

Susunod ay magde-define tayo ng isang function at ang mga parameter ng function na iyon. Gagamitin lang natin ang isang function dito na tinatawag na `search_courses` ngunit maaari kang gumawa ng maraming function.

**Mahalaga** : Ang mga function ay kasama sa system message sa LLM at isasama sa dami ng mga token na mayroon kang magagamit.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Mga Kahulugan** 

Ang istruktura ng depinisyon ng function ay may maraming antas, bawat isa ay may sariling mga katangian. Narito ang isang paghahati-hati ng naka-nest na istruktura:

**Mga Katangian ng Pangunahing Antas ng Function:**

`name` -  Ang pangalan ng function na nais nating tawagin. 

`description` - Ito ang paglalarawan kung paano gumagana ang function. Mahalaga dito na maging tiyak at malinaw. 

`parameters` - Isang listahan ng mga halaga at format na nais mong iproduce ng modelo sa kanyang tugon. 

**Mga Katangian ng Parameters Object:**

`type` -  Ang uri ng data ng parameters object (karaniwang "object")

`properties` - Listahan ng mga tiyak na halaga na gagamitin ng modelo para sa kanyang tugon. 

**Mga Indibidwal na Katangian ng Parameter:**

`name` - Implicit na tinukoy ng property key (halimbawa, "role", "product", "level")

`type` - Ang uri ng data ng partikular na parameter na ito (halimbawa, "string", "number", "boolean") 

`description` - Paglalarawan ng partikular na parameter. 

**Opsyonal na Mga Katangian:**

`required` - Isang array na naglilista kung aling mga parameter ang kinakailangan upang makumpleto ang pagtawag sa function. 


### Paggawa ng tawag sa function  
Pagkatapos idefine ang isang function, kailangan na natin itong isama sa tawag sa Chat Completion API. Ginagawa natin ito sa pamamagitan ng pagdagdag ng `functions` sa request. Sa kasong ito `functions=functions`. 

Mayroon ding opsyon na itakda ang `function_call` sa `auto`. Ibig sabihin nito ay hahayaan natin ang LLM na magdesisyon kung aling function ang dapat tawagin base sa mensahe ng user sa halip na tayo ang mag-assign nito.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Ngayon tingnan natin ang tugon at kung paano ito nakaayos:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Makikita mo na ang pangalan ng function ay tinawag at mula sa mensahe ng user, nagawa ng LLM na hanapin ang datos upang umangkop sa mga argumento ng function.


## 3.Pagsasama ng Mga Tawag sa Function sa Isang Aplikasyon. 


Pagkatapos nating subukan ang na-format na tugon mula sa LLM, maaari na nating isama ito sa isang aplikasyon. 

### Pamamahala ng daloy 

Upang maisama ito sa ating aplikasyon, gawin natin ang mga sumusunod na hakbang: 

Una, gawin natin ang tawag sa mga serbisyo ng OpenAI at itago ang mensahe sa isang variable na tinatawag na `response_message`. 


In [ ]:
response_message = response.choices[0].message

Ngayon ay ide-define natin ang function na tatawag sa Microsoft Learn API upang makakuha ng listahan ng mga kurso:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Bilang pinakamahusay na kasanayan, titingnan muna natin kung nais ng modelo na tumawag ng isang function. Pagkatapos nito, gagawa tayo ng isa sa mga available na function at itutugma ito sa function na tinatawag.  
Pagkatapos, kukunin natin ang mga argumento ng function at itutugma ang mga ito sa mga argumento mula sa LLM.

Sa huli, idaragdag natin ang mensahe ng pagtawag ng function at ang mga halagang ibinalik ng `search_courses` na mensahe. Ito ay nagbibigay sa LLM ng lahat ng impormasyong kailangan nito upang
tumugon sa user gamit ang natural na wika.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Ngayon ay ipapadala natin ang na-update na mensahe sa LLM upang makatanggap tayo ng tugon sa natural na wika sa halip na tugon na naka-format sa API JSON.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code Challenge 

Magaling na trabaho! Upang ipagpatuloy ang iyong pag-aaral ng OpenAI Function Calling maaari kang gumawa: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Higit pang mga parameter ng function na maaaring makatulong sa mga nag-aaral na makahanap ng mas maraming kurso. Maaari mong makita ang mga available na API parameter dito: 
 - Gumawa ng isa pang function call na kumukuha ng higit pang impormasyon mula sa nag-aaral tulad ng kanilang katutubong wika 
 - Gumawa ng error handling kapag ang function call at/o API call ay hindi nagbalik ng anumang angkop na kurso 


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Paalala**:
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagamat nagsusumikap kami para sa katumpakan, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o di-tumpak na impormasyon. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na pangunahing sanggunian. Para sa mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
